In [2]:
import os
import sys
""" module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
sys.path.append(module_path) """
import warnings
import torch
import torch as tc
from torch.func import jacfwd, vmap, grad,jacrev

from Model import Model
from matrixM import MatrixM
from vecF import VecF
from PDE import KdV
import numpy as np
import scipy.integrate 

## Verification of the Matrix

In [3]:
tc.manual_seed(0)
d = 1
m = 2
s=2*m+d*m 
theta=tc.rand(s,requires_grad=True)
pde = KdV(T = 4, xmin=-10, xmax=20, numberTimes=300)

#print(theta)
NN = Model(d,m,theta)
#samples = torch.rand(10, d)
samples=tc.tensor([[0.0223],[0.1689],[0.2939],[0.5185],[0.6977],[0.8000],[0.1610],[0.2823],[0.6816],[0.9152]])
result_grad = NN.grad_theta(theta, samples[0])
result_out = NN.out_prod(theta, samples)

print(result_out)

tensor([[ 0.9603,  0.9126,  0.0318,  0.1655, -0.0613, -0.1496],
        [ 0.9126,  0.8703,  0.0289,  0.1494, -0.0545, -0.1330],
        [ 0.0318,  0.0289,  0.0018,  0.0096, -0.0038, -0.0092],
        [ 0.1655,  0.1494,  0.0096,  0.0521, -0.0204, -0.0499],
        [-0.0613, -0.0545, -0.0038, -0.0204,  0.0086,  0.0210],
        [-0.1496, -0.1330, -0.0092, -0.0499,  0.0210,  0.0513]])


In [4]:
vect_gradM =lambda theta,samples:vmap(NN.grad_theta,in_dims=(None,0))(theta,samples)
Matric=vect_gradM(theta,samples)
Out_Matrix=tc.zeros(len(samples),len(theta),len(theta))
for k in range(len(samples)):
    Out_Matrix[k]=tc.outer(Matric[k],Matric[k])
print(Out_Matrix.mean(dim=0))


tensor([[ 0.9603,  0.9126,  0.0318,  0.1655, -0.0613, -0.1496],
        [ 0.9126,  0.8703,  0.0289,  0.1494, -0.0545, -0.1330],
        [ 0.0318,  0.0289,  0.0018,  0.0096, -0.0038, -0.0092],
        [ 0.1655,  0.1494,  0.0096,  0.0521, -0.0204, -0.0499],
        [-0.0613, -0.0545, -0.0038, -0.0204,  0.0086,  0.0210],
        [-0.1496, -0.1330, -0.0092, -0.0499,  0.0210,  0.0513]],
       grad_fn=<MeanBackward1>)


## Verification of Vector

In [5]:
""" vect_grad =lambda theta,samples:vmap(NN.grad_theta,in_dims=(None,0))(theta,samples).T
vect_result=vect_grad(theta,samples)
print(vect_result.shape)
F=pde.f(1,samples.squeeze(1),NN)
print(F.shape)
Re=vect_result*F
print(Re.mean(dim=1)) """

' vect_grad =lambda theta,samples:vmap(NN.grad_theta,in_dims=(None,0))(theta,samples).T\nvect_result=vect_grad(theta,samples)\nprint(vect_result.shape)\nF=pde.f(1,samples.squeeze(1),NN)\nprint(F.shape)\nRe=vect_result*F\nprint(Re.mean(dim=1)) '

In [6]:
print(samples)
print(theta)

tensor([[0.0223],
        [0.1689],
        [0.2939],
        [0.5185],
        [0.6977],
        [0.8000],
        [0.1610],
        [0.2823],
        [0.6816],
        [0.9152]])
tensor([0.4963, 0.7682, 0.0885, 0.1320, 0.3074, 0.6341], requires_grad=True)


In [7]:
t=1
VF=VecF(pde.d,NN.m,pde.f,samples)
Vect_F=VF(t,theta)
print(Vect_F)


tensor([ 0.9788,  0.8870,  0.0554,  0.3012, -0.1169, -0.2855])


In [8]:
pde.f(t,theta,NN)

tensor([ 1.3149,  1.9214, -0.1390,  0.0287,  0.6905,  1.6718],
       grad_fn=<SubBackward0>)

In [11]:
def function_f(t,y):
    return y.dot(y)+t*y.dot(y**2)
t=0.2
y=tc.rand(3)
Y=function_f(t,y).numpy()
print(Y.dtype)

float32
